<a href="https://colab.research.google.com/github/783atulpatel/Colab_REPO/blob/main/CaseStudy1-MovieLense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




### Case Study : MovieLens 100K - Most Popular Movies
**Dataset Source:**
MovieLens 100K Dataset

**Task:**
Identify the top 10 most popular movies based on the number of ratings.




# Install Java Development Kit (JDK

In [2]:

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# setup Hadoop as told in previous class

In [4]:
!tar -xzf /content/drive/MyDrive/hadoop-setup/hadoop-3.3.6.tar.gz -C /content/

# setting up environment

In [5]:
# Set Hadoop environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/content/hadoop-3.3.6"
os.environ["PATH"] = os.environ["PATH"] + ":" + os.environ["HADOOP_HOME"] + "/bin" + ":" + os.environ["HADOOP_HOME"] + "/sbin"

# Configure Hadoop (replace with your desired configuration)
!echo "export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64" >> ~/.bashrc
!echo "export HADOOP_HOME=/content/hadoop-3.3.6" >> ~/.bashrc
!echo "export PATH=$PATH:$HADOOP_HOME/bin:$HADOOP_HOME/sbin" >> ~/.bashrc

# Source the bashrc file to apply the changes
!source ~/.bashrc

# Verify Hadoop installation
!hadoop version

Hadoop 3.3.6
Source code repository https://github.com/apache/hadoop.git -r 1be78238728da9266a4f88195058f08fd012bf9c
Compiled by ubuntu on 2023-06-18T08:22Z
Compiled on platform linux-x86_64
Compiled with protoc 3.7.1
From source with checksum 5652179ad55f76cb287d9c633bb53bbd
This command was run using /content/hadoop-3.3.6/share/hadoop/common/hadoop-common-3.3.6.jar


# verfying the working of haddop

In [6]:
!jps

8597 Jps


In [7]:
# Create a directory in HDFS
!hdfs dfs -mkdir /content/new_folder

# Getting a movieslens dataset to anlyze with Hadoop

I use a dataset from grouplens. You can get other in:
http://files.grouplens.org/datasets/





In [9]:
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip

--2025-01-04 09:47:42--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  5.56MB/s    in 0.8s    

2025-01-04 09:47:43 (5.56 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]



### To use data extract files. I extract files in path later of -d in command:

In [10]:
!unzip "/content/ml-100k.zip" -d "/content/new_folder"

Archive:  /content/ml-100k.zip
   creating: /content/new_folder/ml-100k/
  inflating: /content/new_folder/ml-100k/allbut.pl  
  inflating: /content/new_folder/ml-100k/mku.sh  
  inflating: /content/new_folder/ml-100k/README  
  inflating: /content/new_folder/ml-100k/u.data  
  inflating: /content/new_folder/ml-100k/u.genre  
  inflating: /content/new_folder/ml-100k/u.info  
  inflating: /content/new_folder/ml-100k/u.item  
  inflating: /content/new_folder/ml-100k/u.occupation  
  inflating: /content/new_folder/ml-100k/u.user  
  inflating: /content/new_folder/ml-100k/u1.base  
  inflating: /content/new_folder/ml-100k/u1.test  
  inflating: /content/new_folder/ml-100k/u2.base  
  inflating: /content/new_folder/ml-100k/u2.test  
  inflating: /content/new_folder/ml-100k/u3.base  
  inflating: /content/new_folder/ml-100k/u3.test  
  inflating: /content/new_folder/ml-100k/u4.base  
  inflating: /content/new_folder/ml-100k/u4.test  
  inflating: /content/new_folder/ml-100k/u5.base  
  inflat

# Installing dependencies to use Python

We can install dependency to use MapReduce using:


In [11]:
!pip install mrjob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 11.9 MB/s eta 0:00:00


### Creating process to use with MRJOB using Python"

To create job in Python, you must see structure of dataset to configure jobs.

In [15]:
!head /content/new_folder/ml-100k/u.data -n 10

196	242	3	881250949
186	302	3	891717742
22	377	1	878887116
244	51	2	880606923
166	346	1	886397596
298	474	4	884182806
115	265	2	881171488
253	465	5	891628467
305	451	3	886324817
6	86	3	883603013


As we can observe

- First column reference to userID.
- Second column reference to movieID.
- Third column reference to rating.
- Fourth column reference to timestamp.

In [18]:
# import modules
%%writefile ratingbreakdown.py
from mrjob.job import MRJob
from mrjob.step import MRStep

class RatingBreakdown(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_ratings,
                   reducer=self.reducer_count_ratings)
        ]

    def mapper_get_ratings(self, _, line):
        # Each line is of the format: user_id\titem_id\trating\ttimestamp
        fields = line.split('\t')
        if len(fields) == 4:
            rating = fields[2]
            yield rating, 1

    def reducer_count_ratings(self, rating, counts):
        yield rating, sum(counts)

if __name__ == "__main__":
    RatingBreakdown.run()


Writing ratingbreakdown.py


### 12th - Running the process

Here we run the process specifing some parameters:
- Python file program  'RatingBreakdown.

 run process,


In [20]:
!python3 ratingbreakdown.py   /content/new_folder/ml-100k/u.data  > /content/new_folder/rating_breakdown_output.txt


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/ratingbreakdown.root.20250104.110315.090921
Running step 1 of 1...
job output is in /tmp/ratingbreakdown.root.20250104.110315.090921/output
Streaming final output from /tmp/ratingbreakdown.root.20250104.110315.090921/output...
Removing temp directory /tmp/ratingbreakdown.root.20250104.110315.090921...


# Viewing the Output


In [21]:
!cat /content/new_folder/rating_breakdown_output.txt

"5"	21201
"1"	6110
"2"	11370
"3"	27145
"4"	34174


This will display the breakdown of ratings from the MovieLens dataset.